In [0]:
#!/usr/bin/env python
from __future__ import print_function
__author__ = 'Tony Beltramelli - www.tonybeltramelli.com'


class Node:
    def __init__(self, key, parent_node, content_holder):
        self.key = key
        self.parent = parent_node
        self.children = []
        self.content_holder = content_holder

    def add_child(self, child):
        self.children.append(child)

    def show(self):
        for child in self.children:
            child.show()

    def render(self, mapping, rendering_function=None):
        content = ""
        for child in self.children:
            placeholder = child.render(mapping, rendering_function)
            if placeholder is None:
                self = None
                return
            else:
                content += placeholder
            
        value = mapping.get(self.key, None)
        if value is None:
            self = None
            return None
        if rendering_function is not None:
            value = rendering_function(self.key, value)

        if len(self.children) != 0:
            value = value.replace(self.content_holder, content)

        return value


In [0]:
import string
import random

def get_random_text(length_text=10, space_number=1, with_upper_case=True):
        results = []
        while len(results) < length_text:
            char = random.choice(string.ascii_letters[:26])
            results.append(char)
        if with_upper_case:
            results[0] = results[0].upper()

        current_spaces = []
        while len(current_spaces) < space_number:
            space_pos = random.randint(2, length_text - 3)
            if space_pos in current_spaces:
                break
            results[space_pos] = " "
            if with_upper_case:
                results[space_pos + 1] = results[space_pos - 1].upper()

            current_spaces.append(space_pos)

        return ''.join(results)
      
def render_content_with_text(key, value):
    if FILL_WITH_RANDOM_TEXT:
        if key.find("btn") != -1:
            value = value.replace(TEXT_PLACE_HOLDER, get_random_text())
        elif key.find("title") != -1:
            value = value.replace(TEXT_PLACE_HOLDER, get_random_text(length_text=5, space_number=0))
        elif key.find("text") != -1:
            value = value.replace(TEXT_PLACE_HOLDER,
                                  get_random_text(length_text=56, space_number=7, with_upper_case=False))
       ## elif key.kind("row")
    return value
#tony beltramelli functions

In [0]:
FILL_WITH_RANDOM_TEXT = True
TEXT_PLACE_HOLDER = "[]"

class Compiler:
  def __init__(self,dsl_file):
    self.dsl_mapping = dsl_file
    self.opening_tag = dsl_mapping["opening-tag"]
    self.closing_tag = dsl_mapping["closing-tag"]
    self.content_holder = self.opening_tag + self.closing_tag
    
  def parseTokens(self, tokens):
    #print(tokens)
    dsl = tokens
    dsl = dsl[1:-1]
    dsl.insert(0,"header")
    
    return dsl
    
  def compile(self, tokens, output_file_path, predict=False):
    dsl_file = self.parseTokens(tokens)
    print(dsl_file)
    dummyNode  = Node("Dummy", None, self.content_holder)
    root = Node("body", dummyNode, self.content_holder)
    #header = Node("header", root, self.content_holder)
    current_parent = root
    
    for token in dsl_file:
      #print(current_parent.key)
      token = token.replace(" ", "")
      token.replace("\n", "")
      token = str(token)
      
      if token == "{":
        #print(token)
        token = token.replace(self.opening_tag,"")
        current_parent = previous
      elif token == "}":
        print(token)
        current_parent = current_parent.parent
      else:
        if token == ",":
          continue
        else:
          element = Node(token, current_parent, self.content_holder)
          current_parent.add_child(element)
      previous = element
    
    output_html = root.render(self.dsl_mapping,rendering_function=render_content_with_text)
    if output_html is None:
      return "still notthis"
    
    with open(output_file_path,'w') as output_file:
      output_file.write(output_html)
      
    return output_html
    
    
    
    
    

In [4]:
from os import listdir
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from tqdm import tqdm
import numpy as np
import h5py as h5py


Using TensorFlow backend.


In [5]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#helper function: read file and return txt as a string
def load_doc(filename):
    with open(filename,'r') as file:
      text = file.read()
      file.close()
    return text

def load_data():
    text = []
    images = []
    #load all files + sort them
    all_filenames = listdir("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/eval/eval3")
    all_filenames.sort()
    for filename in (all_filenames)[-2:]:
        if filename[-3:]=="npz":
            #load imgs already prepped in arr
            image = np.load("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/eval/eval3/"+filename)
            images.append(image['features'])
        else:
            #load bootstrap tokens LABELS/TARGETS
            syntax = '<START>' + load_doc("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/eval/eval3/"+filename) + '<END'
            syntax = ' '.join(syntax.split())
            syntax = syntax.replace(',',' ,')
            #print(syntax)
            text.append(syntax)
            
    images = np.array(images,dtype=float)
   # image_File = np.save('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/image_file',images)
   # text_file = np.save('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/text_fie',text)
    return images,text
  

In [0]:
tokenizer = Tokenizer(filters='',split=" ",lower=False)
tokenizer.fit_on_texts([load_doc('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/resources/bootstrap.vocab')])

train_features,texts = load_data()

In [69]:
with open('/content/drive/My Drive/Colab Notebooks/ScreenshotTest/modelGRU.json','r') as json_file:
  model_json = json_file.read()
  json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("/content/drive/My Drive/Colab Notebooks/ScreenshotTest/weightsGRU.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
#map int to word
def word_for_id(integer,tokenizer):
  for word, index in tokenizer.word_index.items():
    if index == integer:
      return word
  return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    photo = np.array([photo])
    # seed the generation process
    in_text = '<START>header { '
    # iterate over the whole length of the sequence
    print('\nPrediction---->\n\n<START>header { ', end='')
    for i in range(150):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = loaded_model.predict([photo, sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += word + ' '
        # stop if we predict the end of the sequence
        print(word + ', ', end='')
        if word == '<END>':
            break
    return in_text

max_length = 48 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    actualscore2 = list()
    # step over the whole set
    for i in range(len(texts)):
        yhat = generate_desc(model, tokenizer, photos[i], max_length)
        # store actual and predicted
        print('\n\nReal---->\n\n' + texts[i])
        actual.append(texts[i].split())
        actualscore2.append([texts[i].split()])
        predicted.append(yhat.split())
    # calculate BLEU score
    bleu = corpus_bleu(actualscore2, predicted)
    return bleu, actual, predicted, actualscore2

In [74]:
bleu, actual, predicted,score = evaluate_model(loaded_model, texts, train_features, tokenizer, max_length)


Prediction---->

<START>header { btn-inactive, ,, btn-inactive, ,, btn-inactive, ,, btn-inactive, ,, btn-active, }, row, {, double, {, small-title, ,, text, ,, btn-red, }, double, {, small-title, ,, text, ,, btn-red, }, }, row, {, single, {, small-title, ,, text, ,, btn-orange, }, }, row, {, quadruple, {, small-title, ,, text, ,, btn-red, }, quadruple, {, small-title, ,, text, ,, btn-orange, }, quadruple, {, small-title, ,, text, ,, btn-green, }, quadruple, {, small-title, ,, text, ,, btn-green, }, }, row, {, single, {, small-title, ,, text, ,, btn-red, }, }, row, {, single, {, small-title, ,, text, ,, btn-red, }, }, row, {, quadruple, {, small-title, ,, text, ,, btn-red, }, quadruple, {, small-title, ,, text, ,, btn-orange, }, quadruple, {, small-title, ,, text, ,, btn-green, }, quadruple, {, small-title, ,, text, ,, btn-green, }, }, row, {, single, {, small-title, ,, text, ,, btn-orange, }, }, row, {, single, {, small-title, ,, text, 

Real---->

<START>header { btn-inactive , btn-i

In [75]:
end = '.html'

import json

dsl_path = '/content/drive/My Drive/Colab Notebooks/ScreenshotTest/compiler/assets/web-dsl-mapping.json'

with open(dsl_path) as data_file:
  dsl_mapping = json.load(data_file)
compiler = Compiler(dsl_mapping)

directory = '/content/drive/My Drive/Colab Notebooks/ScreenshotTest/html_output/'
actual_website = compiler.compile(actual[0],directory + "actual" + end)
compiled_website = compiler.compile(predicted[0],directory+"compile"+end)

['header', '{', 'btn-inactive', ',', 'btn-inactive', ',', 'btn-inactive', ',', 'btn-inactive', ',', 'btn-active', '}', 'row', '{', 'double', '{', 'small-title', ',', 'text', ',', 'btn-red', '}', 'double', '{', 'small-title', ',', 'text', ',', 'btn-red', '}', '}', 'row', '{', 'single', '{', 'small-title', ',', 'text', ',', 'btn-red', '}', '}', 'row', '{', 'quadruple', '{', 'small-title', ',', 'text', ',', 'btn-red', '}', 'quadruple', '{', 'small-title', ',', 'text', ',', 'btn-green', '}', 'quadruple', '{', 'small-title', ',', 'text', ',', 'btn-green', '}', 'quadruple', '{', 'small-title', ',', 'text', ',', 'btn-orange', '}', '}']
}
}
}
}
}
}
}
}
}
}
}
['header', '{', 'btn-inactive', ',', 'btn-inactive', ',', 'btn-inactive', ',', 'btn-inactive', ',', 'btn-active', '}', 'row', '{', 'double', '{', 'small-title', ',', 'text', ',', 'btn-red', '}', 'double', '{', 'small-title', ',', 'text', ',', 'btn-red', '}', '}', 'row', '{', 'single', '{', 'small-title', ',', 'text', ',', 'btn-orange', '}'

In [76]:
print(actual_website)

<html>
  <header>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap-theme.min.css" integrity="sha384-rHyoN1iRsVXV4nD0JutlnGaslCJuC7uwjduW9SVrLvRYooPp2bWYgmgJQIXwl/Sp" crossorigin="anonymous">
<style>
.header{margin:20px 0}nav ul.nav-pills li{background-color:#333;border-radius:4px;margin-right:10px}.col-lg-3{width:24%;margin-right:1.333333%}.col-lg-6{width:49%;margin-right:2%}.col-lg-12,.col-lg-3,.col-lg-6{margin-bottom:20px;border-radius:6px;background-color:#f5f5f5;padding:20px}.row .col-lg-3:last-child,.row .col-lg-6:last-child{margin-right:0}footer{padding:20px 0;text-align:center;border-top:1px solid #bbb}
</style>
    <title>Scaffol

In [77]:
print(bleu)

0.5016199602939749
